In [1]:
import logging

import numpy as np
from matplotlib import pyplot as plt

from exhbma import (
    ExhaustiveLinearRegression,
    BetaDistributionParams,
    gamma,
    StandardScaler,
    feature_posterior,
    weight_diagram,
    sigma_posterior,
    __version__,
)

In [2]:
log_level = logging.INFO
logger = logging.getLogger(__name__)
logger.propagate = False
handler = logging.StreamHandler()
handler_format = logging.Formatter(
    fmt="%(levelname)s %(asctime)s [%(name)s]: %(message)s",
    datefmt="%Y-%m-%dT%H:%M:%S%z",
)
handler.setFormatter(handler_format)
handler.setLevel(log_level)
logger.setLevel(log_level)
logger.addHandler(handler)

# Generate dataset

In [3]:
n_data, n_features = 50, 10
sigma_noise = 0.1
n_test = 10**3

np.random.seed(0)
X = np.random.randn(n_data, n_features)
nonzero_w = [1, 1, -0.8, 0.5]
w = nonzero_w + [0] * (n_features - len(nonzero_w))
y = np.dot(X, w) + sigma_noise * np.random.randn(n_data)
test_X = np.random.randn(n_test, n_features)
test_y = np.dot(test_X, w) + sigma_noise * np.random.randn(n_test)

# Model training

In [ ]:
n_sigma_points = 20
sigma_noise_min_order = -2.5
sigma_noise_max_order = 0.5
sigma_coef_min_order = -2
sigma_coef_max_order = 1
gamma_shape = 1e-3
gamma_scale = 1e3
fixed_alpha = 0.5


# Data preprocessing
x_scaler = StandardScaler(n_dim=2)
y_scaler = StandardScaler(n_dim=1, scaling=False)
x_scaler.fit(X)
y_scaler.fit(y)
X = x_scaler.transform(X)
y = y_scaler.transform(y)

# Model fitting
reg = ExhaustiveLinearRegression(
    sigma_noise_points=gamma(
        np.logspace(sigma_noise_min_order, sigma_noise_max_order, n_sigma_points),
        low=10 ** sigma_noise_min_order,
        high=10 ** sigma_noise_max_order,
        shape=gamma_shape,
        scale=gamma_scale,
    ),
    sigma_coef_points=gamma(
        np.logspace(sigma_coef_min_order, sigma_coef_max_order, n_sigma_points),
        low=10 ** sigma_coef_min_order,
        high=10 ** sigma_coef_max_order,
        shape=gamma_shape,
        scale=gamma_scale,
    ),
    alpha_params=fixed_alpha,
)
reg.fit(X, y)

  8%|▊         | 81/1023 [00:03<00:37, 25.28it/s]

# Result

## Coefficient

In [ ]:
for i, c in enumerate(reg.coef_):
    logger.info(f"Coefficient of feature {i}: {c:.4f}")

## RMSE

In [ ]:
pred_y = y_scaler.restore(
    reg.predict(x_scaler.transform(test_X), mode="full")
)
rmse = np.power(test_y - pred_y, 2).mean() ** 0.5
logger.info(f"RMSE for test data: {rmse:.4f}")

## Plot

In [ ]:
columns = [f"feat: {i}" for i in range(n_features)]

### Feature posterior

In [ ]:
fig, ax = feature_posterior(
    model=reg,
    title="Feature Posterior Probability",
    ylabel="Probability",
    xticklabels=columns,
)

### Weight diagram

In [ ]:
fig, ax = weight_diagram(
    model=reg,
    yticklabels=columns,
    title="Weight Diagram",
    cbarlabel="Coefficient",
)

### Sigma posterior

In [ ]:
fig, ax = sigma_posterior(
    model=reg,
    title="Log Posterior over ($\sigma_{w}$, $\sigma_{\epsilon}$)",
    xlabel="$\sigma_{w}$",
    ylabel="$\sigma_{\epsilon}$",
    cbarlabel="Log Likelihood",
)